In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# Path to google drive folder
# %cd /content/drive/MyDrive/Colab Notebooks/Disertasi-Ahmad-Rofiqul/002. Laporan-eksperimen

%cd /content/drive/MyDrive/Colab Notebooks/Disertasi-Ahmad-Rofiqul/002.Experiment_SWI_16052024/

/content/drive/MyDrive/Colab Notebooks/Disertasi-Ahmad-Rofiqul/002.Experiment_SWI_16052024


In [3]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from xgboost import XGBRegressor, DMatrix, cv
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, classification_report, ConfusionMatrixDisplay, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
import xgboost as xgb

In [4]:
!ls

'010_01_Agriculture_N2O_IF_DBSCAN_OneClassSVM_Ensemble_OUTLIERS_DETECTION_Uji Performance.ipynb'
'010_02_Agriculture_N2O_IF_DBSCAN_OneClassSVM_Ensemble_OUTLIERS_DETECTION_Uji Performance_consensus_scoring.ipynb'
'010_04_Agriculture_N2O_IF_DBSCAN_OneClassSVM_Ensemble_OUTLIERS_DETECTION_weighted_voting (iA).ipynb'
'011_01_Agriculture_N2O_IF_DBSCAN_OneClassSVM_Ensemble_OUTLIERS_DETECTION_Uji Performance 13092024.ipynb'
 01.Dataset-Rofiqul
 09_03_Agriculture_N2O_IF_DBSCAN_OneClassSVM_Ensemble_NOIQR_OUTLIERS_DETECTION_UPDATE24052024.ipynb
 09_03_Agriculture_N2O_IF_DBSCAN_OneClassSVM_Ensemble_OUTLIERS_DETECTION_update23052024.ipynb
 09_03_Agriculture_N2O_IF_DBSCAN_OneClassSVM_Ensemble_+_XGBoost.ipynb
'09_03_Agriculture_N2O_IF_DBSCAN_OneClassSVM_Ensemble_+_XGBoost_SWI(1).ipynb'
 09_03_Agriculture_N2O_IF_DBSCAN_OneClassSVM_Ensemble_+_XGBoost_SWI_Update21052024.ipynb
 09_04_1_Agricultural_N2O_IDO_Training_Testing_Plain_SWI_22052024.ipynb
 09_04_1_Agricultural_N2O_IDO_Training_Testing_Plain_SWI_

In [5]:
# Load the dataset
file_path = '01.Dataset-Rofiqul/agriculture_dataset.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset for review
data.head(),data.info(),data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2246 entries, 0 to 2245
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         2246 non-null   object 
 1   Year         2246 non-null   int64  
 2   Experiment   2246 non-null   object 
 3   DataUse      2246 non-null   object 
 4   Replication  2246 non-null   object 
 5   Month        2246 non-null   object 
 6   Vegetation   2246 non-null   object 
 7   VegType      2246 non-null   object 
 8   N2O          2246 non-null   float64
 9   N_rate       2246 non-null   int64  
 10  PP2          2246 non-null   float64
 11  PP7          2246 non-null   float64
 12  AirT         2246 non-null   float64
 13  DAF_TD       2246 non-null   int64  
 14  DAF_SD       2246 non-null   int64  
 15  WFPS25cm     2194 non-null   float64
 16  NH4          2170 non-null   float64
 17  NO3          2216 non-null   float64
 18  Clay         2246 non-null   float64
 19  Sand  

(      Date  Year Experiment   DataUse Replication     Month Vegetation  \
 0   2/9/12  2012   BCSE_KBS  Building          R1  February       Corn   
 1  2/10/12  2012   BCSE_KBS  Building          R1  February       Corn   
 2  2/18/12  2012   BCSE_KBS  Building          R1  February       Corn   
 3  2/19/12  2012   BCSE_KBS  Building          R1  February       Corn   
 4  3/16/12  2012   BCSE_KBS  Building          R1     March       Corn   
 
   VegType       N2O  N_rate  ...   PP7  AirT  DAF_TD  DAF_SD  WFPS25cm  \
 0  Annual  3.896742     170  ...  0.00  -2.0     276     241  0.666508   
 1  Annual  2.190218     170  ...  0.00  -2.4     277     242  0.640608   
 2  Annual  3.542594     170  ...  8.64   0.3     285     250  0.728085   
 3  Annual  3.342870     170  ...  8.13  -3.8     286     251  0.686872   
 4  Annual  2.947778     170  ...  8.39  17.6     312     277  0.716221   
 
          NH4        NO3  Clay   Sand       SOM  
 0  11.046340  22.940812  62.5  637.5  1.17407

## Detect Nilai NaN

In [6]:

missing_values_count = data.isna().sum()
missing_values_count

,0
Date,0
Year,0
Experiment,0
DataUse,0
Replication,0
Month,0
Vegetation,0
VegType,0
N2O,0
N_rate,0


## Impute NaN or missing value with Mean

In [7]:
# List of columns to impute missing values with the mean
columns_to_impute = ['N2O', 'N_rate', 'AirT', 'WFPS25cm', 'NO3', 'NH4', 'Clay', 'Sand']

# Loop through each column and fill NaN with the mean of the respective column
for column in columns_to_impute:
    data.loc[:, column] = data[column].fillna(data[column].mean())


# Verify that there are no missing values left
missing_values_after_imputation = data.isna().sum()
missing_values_after_imputation


,0
Date,0
Year,0
Experiment,0
DataUse,0
Replication,0
Month,0
Vegetation,0
VegType,0
N2O,0
N_rate,0


In [8]:
# Save the cleaned dataset to a new CSV file
output_file_path = '01.Dataset-Rofiqul/BeforeRemoveOutliers/001.input_NaN_mean_agriculture_dataset.csv'
data.to_csv(output_file_path, index=False)

output_file_path

'01.Dataset-Rofiqul/BeforeRemoveOutliers/001.input_NaN_mean_agriculture_dataset.csv'

## Detect NaN Value

In [9]:
missing_values_count = data.isna().sum()
missing_values_count

,0
Date,0
Year,0
Experiment,0
DataUse,0
Replication,0
Month,0
Vegetation,0
VegType,0
N2O,0
N_rate,0


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2246 entries, 0 to 2245
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         2246 non-null   object 
 1   Year         2246 non-null   int64  
 2   Experiment   2246 non-null   object 
 3   DataUse      2246 non-null   object 
 4   Replication  2246 non-null   object 
 5   Month        2246 non-null   object 
 6   Vegetation   2246 non-null   object 
 7   VegType      2246 non-null   object 
 8   N2O          2246 non-null   float64
 9   N_rate       2246 non-null   int64  
 10  PP2          2246 non-null   float64
 11  PP7          2246 non-null   float64
 12  AirT         2246 non-null   float64
 13  DAF_TD       2246 non-null   int64  
 14  DAF_SD       2246 non-null   int64  
 15  WFPS25cm     2246 non-null   float64
 16  NH4          2246 non-null   float64
 17  NO3          2246 non-null   float64
 18  Clay         2246 non-null   float64
 19  Sand  

## StandartScaler

In [11]:
# Initialize the StandardScaler
scaler = StandardScaler()

# Apply StandardScaler to the numerical columns
numerical_columns = data.select_dtypes(include=['float64', 'int64']).columns
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

# Save the scaled dataset to a new CSV file
scaled_output_file_path = '01.Dataset-Rofiqul/BeforeRemoveOutliers/002.standard_scaler_agriculture_dataset.csv'
data.to_csv(scaled_output_file_path, index=False)

# Display the head of the scaled dataset
scaled_data_head = data.head()
scaled_data_head, scaled_output_file_path

(      Date      Year Experiment   DataUse Replication     Month Vegetation  \
 0   2/9/12  0.020682   BCSE_KBS  Building          R1  February       Corn   
 1  2/10/12  0.020682   BCSE_KBS  Building          R1  February       Corn   
 2  2/18/12  0.020682   BCSE_KBS  Building          R1  February       Corn   
 3  2/19/12  0.020682   BCSE_KBS  Building          R1  February       Corn   
 4  3/16/12  0.020682   BCSE_KBS  Building          R1     March       Corn   
 
   VegType       N2O    N_rate  ...       PP7      AirT    DAF_TD    DAF_SD  \
 0  Annual -0.125408  0.481233  ... -0.811416 -1.518617  0.556855  0.294676   
 1  Annual -0.185324  0.481233  ... -0.811416 -1.560776  0.563758  0.301675   
 2  Annual -0.137842  0.481233  ... -0.447138 -1.276207  0.618974  0.357667   
 3  Annual -0.144855  0.481233  ... -0.468640 -1.708330  0.625877  0.364667   
 4  Annual -0.158726  0.481233  ... -0.457678  0.547142  0.805331  0.546643   
 
    WFPS25cm       NH4       NO3      Clay      

## IQR Detect Outlier for Groundtruth

In [12]:
# """## IQR Outlier Detection and Groundtruth Creation"""

# # Function to calculate IQR bounds
# def iqr_bounds(series):
#     Q1 = series.quantile(0.25)
#     Q3 = series.quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     return lower_bound, upper_bound

# # Detect outliers and create summary table
# summary_table = []
# groundtruth = pd.DataFrame(index=data.index)  # To store outliers

# for col in numerical_columns:
#     lower_bound_IQR, upper_bound_IQR = iqr_bounds(data[col])

#     # Detect outliers
#     outliers_IQR = (data[col] < lower_bound_IQR) | (data[col] > upper_bound_IQR)

#     # Store the outlier information in the 'groundtruth' dataframe
#     groundtruth[col + '_outlier'] = outliers_IQR.astype(int)  # 1 for outlier, 0 for non-outlier

#     # Append information to summary table
#     summary_table.append({
#         'Column': col,
#         'Method': 'IQR',
#         'Lower Bound': lower_bound_IQR,
#         'Upper Bound': upper_bound_IQR,
#         'Outlier Count': outliers_IQR.sum()
#     })

# # Convert the summary table into a DataFrame for review
# summary_df = pd.DataFrame(summary_table)
# print("Outlier Summary IQR:\n", summary_df)

# # Create the Groundtruth column by checking if there is any outlier in any column
# groundtruth['Groundtruth'] = groundtruth.any(axis=1).astype(int)  # 1 if outlier in any column, 0 if not

# # Add the groundtruth to the original data (not scaled)
# data_with_groundtruth = pd.concat([data, groundtruth['Groundtruth']], axis=1)

# # Save the dataset with Groundtruth column
# groundtruth_output_file_path = '01.Dataset-Rofiqul/BeforeRemoveOutliers/003.groundtruth_agriculture_dataset.csv'
# data_with_groundtruth.to_csv(groundtruth_output_file_path, index=False)

# print("Groundtruth dataset saved at:", groundtruth_output_file_path)
# # Example outlier detection using IQR (Interquartile Range)
# Q1 = data['N2O'].quantile(0.25)
# Q3 = data['N2O'].quantile(0.75)
# IQR = Q3 - Q1
# lower_bound = Q1 - 1.5 * IQR
# upper_bound = Q3 + 1.5 * IQR

# # Define GroundTruth based on IQR
# data['GroundTruth'] = np.where((data['N2O'] < lower_bound) | (data['N2O'] > upper_bound), -1, 1)

# # Select features for model input (X) and GroundTruth as the label (y)
# X = data[['N2O', 'N_rate', 'AirT', 'WFPS25cm', 'NO3', 'NH4', 'Clay', 'Sand']]  # Selected features
# y = data['GroundTruth']  # GroundTruth as the label (-1 for outlier, 1 for inlier)

# # Split data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # Verify the shapes of the training and testing sets
# print("X_train shape:", X_train.shape)
# print("y_train shape:", y_train.shape)
# print("X_test shape:", X_test.shape)
# print("y_test shape:", y_test.shape)

# # Save the dataset with the GroundTruth column
# groundtruth_output_file_path = '01.Dataset-Rofiqul/BeforeRemoveOutliers/003.groundtruth_agriculture_dataset.csv'
# data.to_csv(groundtruth_output_file_path, index=False)

# print("Groundtruth dataset saved at:", groundtruth_output_file_path)

# # Create a summary DataFrame to show counts of inliers and outliers
# summary_df = pd.DataFrame({
#     'Inliers (1)': [np.sum(data['GroundTruth'] == 1)],
#     'Outliers (-1)': [np.sum(data['GroundTruth'] == -1)]
# })

# # Display the summary of inliers and outliers
# print("Summary of GroundTruth labels:")
# print(summary_df)

## N2O Only
# Extract the N2O column
N2O_data = data['N2O']

# Detect outliers using IQR method
Q1 = N2O_data.quantile(0.25)
Q3 = N2O_data.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Define GroundTruth based on IQR only for N2O
data['GroundTruth'] = np.where((N2O_data < lower_bound) | (N2O_data > upper_bound), -1, 1)

# Select features for model input (X) and GroundTruth as the label (y)
# X = data[['N_rate', 'AirT', 'WFPS25cm', 'NO3', 'NH4', 'Clay', 'Sand']]  # Use only non-N2O features
X = data[['N2O']]  # Use only non-N2O features
y = data['GroundTruth']  # GroundTruth as the label (-1 for outlier, 1 for inlier)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Verify the shapes of the training and testing sets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

# Save the dataset with the GroundTruth column
groundtruth_output_file_path = '01.Dataset-Rofiqul/BeforeRemoveOutliers/003.groundtruth_agriculture_dataset.csv'
data.to_csv(groundtruth_output_file_path, index=False)

print("Groundtruth dataset saved at:", groundtruth_output_file_path)

# Create a summary DataFrame to show counts of inliers and outliers
summary_df = pd.DataFrame({
    'Inliers (1)': [np.sum(data['GroundTruth'] == 1)],
    'Outliers (-1)': [np.sum(data['GroundTruth'] == -1)]
})

# Display the summary of inliers and outliers
print("Summary of GroundTruth labels:")
print(summary_df)

X_train shape: (1572, 1)
y_train shape: (1572,)
X_test shape: (674, 1)
y_test shape: (674,)
Groundtruth dataset saved at: 01.Dataset-Rofiqul/BeforeRemoveOutliers/003.groundtruth_agriculture_dataset.csv
Summary of GroundTruth labels:
   Inliers (1)  Outliers (-1)
0         1995            251


# Isolation Forest

In [47]:
# Train and predict using Isolation Forest
iso_forest = IsolationForest(contamination=0.06, random_state=42)


iso_forest.fit(X_train)
y_pred_if = iso_forest.predict(X_test)

# Performance Evaluation for Isolation Forest

# Evaluate Isolation Forest using the IQR-based GroundTruth
accuracy_if = accuracy_score(y_test, y_pred_if)
precision_if = precision_score(y_test, y_pred_if, pos_label=-1)
recall_if = recall_score(y_test, y_pred_if, pos_label=-1)
f1_if = f1_score(y_test, y_pred_if, pos_label=-1)

# Display performance metrics for Isolation Forest
print("Performance Metrics for Isolation Forest (using GroundTruth from IQR):")
print(f"Accuracy: {accuracy_if:.2f}")
print(f"Precision: {precision_if:.2f}")
print(f"Recall: {recall_if:.2f}")
print(f"F1-Score: {f1_if:.2f}")

Performance Metrics for Isolation Forest (using GroundTruth from IQR):
Accuracy: 0.94
Precision: 0.83
Recall: 0.49
F1-Score: 0.62


#DBScan

In [46]:
# Train and predict using DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=2)
y_pred_db = dbscan.fit_predict(X_test)

# Convert DBSCAN predictions to binary (outliers = -1, inliers = 1)
y_pred_db_binary = np.where(y_pred_db == -1, -1, 1)

# Evaluate DBSCAN (using binary labels)
accuracy_db = accuracy_score(y_test, y_pred_db_binary)
precision_db = precision_score(y_test, y_pred_db_binary, pos_label=-1)
recall_db = recall_score(y_test, y_pred_db_binary, pos_label=-1)
f1_db = f1_score(y_test, y_pred_db_binary, pos_label=-1)

print(f"DBSCAN           - Accuracy: {accuracy_db:.2f}, Precision: {precision_db:.2f}, Recall: {recall_db:.2f}, F1-Score: {f1_db:.2f}")

DBSCAN           - Accuracy: 0.90, Precision: 1.00, Recall: 0.03, F1-Score: 0.06


# One Class SVM

In [48]:
# Train and predict using One-Class SVM
ocsvm = OneClassSVM(nu=0.05, kernel='rbf', gamma='auto')
# ocsvm = OneClassSVM(nu=0.5, kernel='linear', gamma=0.01)
y_pred_svm = ocsvm.fit_predict(X_test)

# Evaluate One-Class SVM
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm, pos_label=-1)
recall_svm = recall_score(y_test, y_pred_svm, pos_label=-1)
f1_svm = f1_score(y_test, y_pred_svm, pos_label=-1)

print(f"One-Class SVM    - Accuracy: {accuracy_svm:.2f}, Precision: {precision_svm:.2f}, Recall: {recall_svm:.2f}, F1-Score: {f1_svm:.2f}")

One-Class SVM    - Accuracy: 0.92, Precision: 0.72, Recall: 0.38, F1-Score: 0.50


# Display Performance for each model

In [49]:
# Display performance metrics for each model
print("Performance Metrics for Individual Models:")
print(f"Isolation Forest - Accuracy: {accuracy_if:.2f}, Precision: {precision_if:.2f}, Recall: {recall_if:.2f}, F1-Score: {f1_if:.2f}")
print(f"DBSCAN           - Accuracy: {accuracy_db:.2f}, Precision: {precision_db:.2f}, Recall: {recall_db:.2f}, F1-Score: {f1_db:.2f}")
print(f"One-Class SVM    - Accuracy: {accuracy_svm:.2f}, Precision: {precision_svm:.2f}, Recall: {recall_svm:.2f}, F1-Score: {f1_svm:.2f}")

Performance Metrics for Individual Models:
Isolation Forest - Accuracy: 0.94, Precision: 0.83, Recall: 0.49, F1-Score: 0.62
DBSCAN           - Accuracy: 0.90, Precision: 1.00, Recall: 0.03, F1-Score: 0.06
One-Class SVM    - Accuracy: 0.92, Precision: 0.72, Recall: 0.38, F1-Score: 0.50


# Ensemble (Manual)

In [50]:
# Combine the predictions using majority voting

# Collect predictions from all models
predictions = np.vstack([y_pred_if, y_pred_db_binary, y_pred_svm]).T

# Majority voting (mode of predictions)
y_pred_ensemble_majority = stats.mode(predictions, axis=1)[0].flatten()

# Evaluate the majority voting ensemble's performance
accuracy_majority = accuracy_score(y_test, y_pred_ensemble_majority)
error_rate_majority = 1 - accuracy_majority
precision_majority = precision_score(y_test, y_pred_ensemble_majority, pos_label=-1)
recall_majority = recall_score(y_test, y_pred_ensemble_majority, pos_label=-1)
f1_majority = f1_score(y_test, y_pred_ensemble_majority, pos_label=-1)

print("\nPerformance Metrics for Majority Voting Ensemble:")
print(f"Majority Voting - Accuracy: {accuracy_majority:.2f}, Error Rate: {error_rate_majority:.2f}, Precision: {precision_majority:.2f}, Recall: {recall_majority:.2f}, F1-Score: {f1_majority:.2f}")

# Weighted Voting
# Assign weights to the models (you can adjust these based on individual model performance)
# weights = [0.5, 0.1, 0.4]  # Example: 40% for Isolation Forest, 30% for DBSCAN, 30% for One-Class SVM
weights = [0.4, 0.3, 0.3]  # Example: 40% for Isolation Forest, 30% for DBSCAN, 30% for One-Class SVM

# Convert the predictions from (-1, 1) to (0, 1) for easier summing
predictions_binary = np.where(predictions == -1, 0, 1)

# Perform weighted voting
weighted_sum = np.dot(predictions_binary, weights)

# Setup threshold
threshold = 0.4

# Convert weighted sum back to (-1, 1) labels: if sum >= 0.5, predict inlier (1); else outlier (-1)
y_pred_ensemble_weighted = np.where(weighted_sum >= threshold, 1, -1)

# Evaluate the weighted voting ensemble's performance
accuracy_weighted = accuracy_score(y_test, y_pred_ensemble_weighted)
error_rate_weighted = 1 - accuracy_weighted
precision_weighted = precision_score(y_test, y_pred_ensemble_weighted, pos_label=-1)
recall_weighted = recall_score(y_test, y_pred_ensemble_weighted, pos_label=-1)
f1_weighted = f1_score(y_test, y_pred_ensemble_weighted, pos_label=-1)

print("\nPerformance Metrics for Weighted Voting Ensemble:")
print(f"Weighted Voting - Accuracy: {accuracy_weighted:.2f}, Error Rate: {error_rate_weighted:.2f}, Precision: {precision_weighted:.2f}, Recall: {recall_weighted:.2f}, F1-Score: {f1_weighted:.2f}")

# Display performance comparison for all models and ensembles
accuracy_if = accuracy_score(y_test, y_pred_if)
error_rate_if = 1 - accuracy_if
accuracy_db = accuracy_score(y_test, y_pred_db_binary)
error_rate_db = 1 - accuracy_db
accuracy_svm = accuracy_score(y_test, y_pred_svm)
error_rate_svm = 1 - accuracy_svm

print("\nPerformance Metrics for Individual Models, Majority Voting, and Weighted Voting:")
print(f"Isolation Forest - Accuracy: {accuracy_if:.2f}, Error Rate: {error_rate_if:.2f}, Precision: {precision_if:.2f}, Recall: {recall_if:.2f}, F1-Score: {f1_if:.2f}")
print(f"DBSCAN           - Accuracy: {accuracy_db:.2f}, Error Rate: {error_rate_db:.2f}, Precision: {precision_db:.2f}, Recall: {recall_db:.2f}, F1-Score: {f1_db:.2f}")
print(f"One-Class SVM    - Accuracy: {accuracy_svm:.2f}, Error Rate: {error_rate_svm:.2f}, Precision: {precision_svm:.2f}, Recall: {recall_svm:.2f}, F1-Score: {f1_svm:.2f}")
print(f"Majority Voting  - Accuracy: {accuracy_majority:.2f}, Error Rate: {error_rate_majority:.2f}, Precision: {precision_majority:.2f}, Recall: {recall_majority:.2f}, F1-Score: {f1_majority:.2f}")
print(f"Weighted Voting  - Accuracy: {accuracy_weighted:.2f}, Error Rate: {error_rate_weighted:.2f}, Precision: {precision_weighted:.2f}, Recall: {recall_weighted:.2f}, F1-Score: {f1_weighted:.2f}")

# # Check if the models are making diverse predictions
# agreement = np.sum(np.all(predictions == predictions[:, [0]], axis=1)) / len(predictions)
# print(f"Percentage of total agreement between all models: {agreement * 100:.2f}%")



Performance Metrics for Majority Voting Ensemble:
Majority Voting - Accuracy: 0.93, Error Rate: 0.07, Precision: 0.79, Recall: 0.38, F1-Score: 0.51

Performance Metrics for Weighted Voting Ensemble:
Weighted Voting - Accuracy: 0.93, Error Rate: 0.07, Precision: 0.79, Recall: 0.38, F1-Score: 0.51

Performance Metrics for Individual Models, Majority Voting, and Weighted Voting:
Isolation Forest - Accuracy: 0.94, Error Rate: 0.06, Precision: 0.83, Recall: 0.49, F1-Score: 0.62
DBSCAN           - Accuracy: 0.90, Error Rate: 0.10, Precision: 1.00, Recall: 0.03, F1-Score: 0.06
One-Class SVM    - Accuracy: 0.92, Error Rate: 0.08, Precision: 0.72, Recall: 0.38, F1-Score: 0.50
Majority Voting  - Accuracy: 0.93, Error Rate: 0.07, Precision: 0.79, Recall: 0.38, F1-Score: 0.51
Weighted Voting  - Accuracy: 0.93, Error Rate: 0.07, Precision: 0.79, Recall: 0.38, F1-Score: 0.51


# XGBoost Prediction

In [26]:
# # --- Prediksi XGBoost ---

# # Fungsi untuk menghitung metrik evaluasi
# def evaluate_model(y_true, y_pred):
#     mse = mean_squared_error(y_true, y_pred)
#     rmse = np.sqrt(mse)
#     mae = mean_absolute_error(y_true, y_pred)
#     r2 = r2_score(y_true, y_pred)
#     return mse, rmse, mae, r2

# # --- Gunakan prediksi weighted voting untuk menentukan data inliers ---
# inliers_mask_test = y_pred_ensemble_weighted == 1  # Inliers adalah yang diprediksi sebagai 1 di X_test

# # Pastikan N2O tidak digunakan sebagai fitur input
# X_inliers_test = X_test[inliers_mask_test].drop(columns=['N2O'])  # Hapus kolom N2O dari fitur input
# y_inliers_test = X_test[inliers_mask_test]['N2O']  # Target adalah kolom N2O

# # --- Split data inliers menjadi training dan testing set untuk model XGBoost ---
# X_train_inliers, X_test_inliers, y_train_inliers, y_test_inliers = train_test_split(X_inliers_test, y_inliers_test, test_size=0.3, random_state=42)

# # --- Inisialisasi dan latih model XGBoost ---
# xg_reg = XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1,
#                       max_depth=5, alpha=10, n_estimators=100)

# xg_reg.fit(X_train_inliers, y_train_inliers)

# # --- Prediksi N2O pada data testing ---
# y_pred_xgb = xg_reg.predict(X_test_inliers)

# # --- Evaluasi hasil prediksi ---
# mse, rmse, mae, r2 = evaluate_model(y_test_inliers, y_pred_xgb)

# # Cetak metrik evaluasi regresi
# print("\nEvaluation Metrics for XGBoost Regressor on N2O Emissions:")
# print(f"R-squared (R2): {r2:.2f}")
# print(f"Mean Absolute Error (MAE): {mae:.2f}")
# print(f"Mean Squared Error (MSE): {mse:.2f}")
# print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

# --- Prediksi XGBoost ---

# Fungsi untuk menghitung metrik evaluasi
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mse, rmse, mae, r2

# --- Gunakan prediksi weighted voting untuk menentukan data inliers ---
inliers_mask_test = y_pred_ensemble_weighted == 1  # Inliers adalah yang diprediksi sebagai 1 di X_test
X_inliers_test = X_test[inliers_mask_test]  # Data yang tidak terdeteksi sebagai outliers di X_test
y_inliers_test = y_test[inliers_mask_test]  # Hanya prediksi N2O untuk data inliers di X_test

# --- Split data inliers menjadi training dan testing set untuk model XGBoost ---
X_train_inliers, X_test_inliers, y_train_inliers, y_test_inliers = train_test_split(X_inliers_test, y_inliers_test, test_size=0.3, random_state=42)

# --- Inisialisasi dan latih model XGBoost ---
xg_reg = XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1,
                      max_depth=5, alpha=10, n_estimators=100)

xg_reg.fit(X_train_inliers, y_train_inliers)

# --- Prediksi N2O pada data testing ---
y_pred_xgb = xg_reg.predict(X_test_inliers)

# --- Evaluasi hasil prediksi ---
mse, rmse, mae, r2 = evaluate_model(y_test_inliers, y_pred_xgb)

# Cetak metrik evaluasi regresi
print("\nEvaluation Metrics for XGBoost Regressor on N2O Emissions:")
print(f"R-squared (R2): {r2:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")



Evaluation Metrics for XGBoost Regressor on N2O Emissions:
R-squared (R2): 0.96
Mean Absolute Error (MAE): 0.04
Mean Squared Error (MSE): 0.01
Root Mean Squared Error (RMSE): 0.07


# KFold Validation

In [27]:
# KFold XGBoost
def kfold_xgboost(X_train, y_train):
    params = {
        'objective': 'reg:squarederror',
        'learning_rate': 0.1,
        'max_depth': 5,
        'alpha': 10,
        'colsample_bytree': 0.3
    }
    dtrain = DMatrix(X_train, label=y_train)

    # 5-fold cross-validation menggunakan num_boost_round
    cv_results = xgb.cv(dtrain=dtrain, params=params, nfold=5, num_boost_round=100,
                        metrics=['rmse', 'mae'], as_pandas=True, seed=42)

    return cv_results

# Prepare an empty dictionary to store results
results = {
    "Test Size": [],
    "CV MSE": [],
    "MSE": [],
    "RMSE": [],
    "MAE": [],
    "R2": []
}

# Simulasi pengujian dengan berbagai test sizes (20%, 25%, 30%, 35%)
test_sizes = [0.2, 0.25, 0.3, 0.35]

for test_size in test_sizes:
    # Split data into train and test sets
    X_train_inliers, X_test_inliers, y_train_inliers, y_test_inliers = train_test_split(X_inliers_test, y_inliers_test, test_size=test_size, random_state=42)

    # Train the XGBoost model
    xg_reg = XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1,
                          max_depth=5, alpha=10, n_estimators=100)
    xg_reg.fit(X_train_inliers, y_train_inliers)

    # Predict the test data
    y_pred_xgb = xg_reg.predict(X_test_inliers)

    # Evaluate predictions
    mse, rmse, mae, r2 = evaluate_model(y_test_inliers, y_pred_xgb)

    # --- KFold Cross-Validation ---
    kfold_results = kfold_xgboost(X_train_inliers, y_train_inliers)
    mse_cv = kfold_results['test-rmse-mean'].values[-1] ** 2
    rmse_cv = np.sqrt(mse_cv)

    # Store the results for this test size
    results["Test Size"].append(f"{int(test_size*100)}%")
    results["CV MSE"].append(round(mse_cv, 4))
    results["MSE"].append(round(mse, 4))
    results["RMSE"].append(round(rmse, 4))
    results["MAE"].append(round(mae, 4))
    results["R2"].append(round(r2, 4))

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Display the DataFrame as a text-based table (easily copyable)
results_df
print("\nPrediction Evaluation with KFold XGBoost")
print(results_df.to_string(index=False))



Prediction Evaluation with KFold XGBoost
Test Size  CV MSE    MSE   RMSE    MAE     R2
      20%  0.0216 0.0060 0.0777 0.0363 0.9664
      25%  0.0138 0.0050 0.0706 0.0347 0.9657
      30%  0.0284 0.0056 0.0750 0.0369 0.9604
      35%  0.0236 0.0056 0.0748 0.0385 0.9597


# Standart Cross Validation

In [28]:
import pandas as pd
from tabulate import tabulate

# Fungsi untuk Standard Cross-Validation dengan cross_val_score
def standard_cross_validation(X_train, y_train):
    xg_reg = XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1,
                          max_depth=5, alpha=10, n_estimators=100)

    # 5-fold cross-validation
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)

    # Mean Squared Error
    mse_scores = cross_val_score(xg_reg, X_train, y_train, cv=kfold, scoring='neg_mean_squared_error')
    mae_scores = cross_val_score(xg_reg, X_train, y_train, cv=kfold, scoring='neg_mean_absolute_error')
    r2_scores = cross_val_score(xg_reg, X_train, y_train, cv=kfold, scoring='r2')

    # Take the mean and convert negative MSE and MAE to positive
    mse_mean = -np.mean(mse_scores)
    rmse_mean = np.sqrt(mse_mean)
    mae_mean = -np.mean(mae_scores)
    r2_mean = np.mean(r2_scores)

    return mse_mean, rmse_mean, mae_mean, r2_mean

# Prepare an empty dictionary to store results
results_standard_cv = {
    "Test Size": [],
    "CV MSE": [],
    "MSE": [],
    "RMSE": [],
    "MAE": [],
    "R²": []
}

# Simulasi pengujian dengan berbagai test sizes (20%, 25%, 30%, 35%)
test_sizes = [0.2, 0.25, 0.3, 0.35]

for test_size in test_sizes:
    # Split data into train and test sets
    X_train_inliers, X_test_inliers, y_train_inliers, y_test_inliers = train_test_split(X_inliers_test, y_inliers_test, test_size=test_size, random_state=42)

    # Train the XGBoost model
    xg_reg = XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1,
                          max_depth=5, alpha=10, n_estimators=100)
    xg_reg.fit(X_train_inliers, y_train_inliers)

    # Predict the test data
    y_pred_xgb = xg_reg.predict(X_test_inliers)

    # Evaluate predictions
    mse, rmse, mae, r2 = evaluate_model(y_test_inliers, y_pred_xgb)

    # --- Standard Cross-Validation ---
    mse_cv, rmse_cv, mae_cv, r2_cv = standard_cross_validation(X_train_inliers, y_train_inliers)

    # Store the results for this test size
    results_standard_cv["Test Size"].appen(f"{int(test_size*100)}%")
    results_standard_cv["CV MSE"].append(round(mse_cv, 4))
    results_standard_cv["MSE"].append(round(mse, 4))
    results_standard_cv["RMSE"].append(round(rmse, 4))
    results_standard_cv["MAE"].append(round(mae, 4))
    results_standard_cv["R²"].append(round(r2, 4))

# Convert results to DataFrame
results_standard_cv_df = pd.DataFrame(results_standard_cv)

# Display the DataFrame as a formatted table
table_str = tabulate(results_standard_cv_df, headers='keys', tablefmt='grid', showindex=False)
print("\nPrediction Evaluation with Standard Cross-Validation")
print(table_str)



Prediction Evaluation with Standard Cross-Validation
+-------------+----------+--------+--------+--------+--------+
| Test Size   |   CV MSE |    MSE |   RMSE |    MAE |     R² |
+=============+==========+========+========+========+========+
| 20%         |   0.0304 | 0.006  | 0.0777 | 0.0363 | 0.9664 |
+-------------+----------+--------+--------+--------+--------+
| 25%         |   0.0143 | 0.005  | 0.0706 | 0.0347 | 0.9657 |
+-------------+----------+--------+--------+--------+--------+
| 30%         |   0.04   | 0.0056 | 0.075  | 0.0369 | 0.9604 |
+-------------+----------+--------+--------+--------+--------+
| 35%         |   0.0271 | 0.0056 | 0.0748 | 0.0385 | 0.9597 |
+-------------+----------+--------+--------+--------+--------+


# GridSearchCV

In [29]:
# Set hyperparameters for tuning
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.6, 0.8]
}

# Simulasi pengujian dengan berbagai test sizes (20%, 25%, 30%, 35%)
test_sizes = [0.2, 0.25, 0.3, 0.35]

# Prepare an empty dictionary to store results
results = {
    "Test Size": [],
    "CV MSE": [],
    "MSE": [],
    "RMSE": [],
    "MAE": [],
    "R²": []
}

for test_size in test_sizes:
    # Split data into train and test sets
    X_train_inliers, X_test_inliers, y_train_inliers, y_test_inliers = train_test_split(X_inliers_test, y_inliers_test, test_size=test_size, random_state=42)

    # Inisialisasi XGBoost Regressor
    xg_reg = XGBRegressor(objective='reg:squarederror')

    # GridSearchCV untuk hyperparameter tuning
    grid_search = GridSearchCV(estimator=xg_reg, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train_inliers, y_train_inliers)

    # Predict with the best model
    best_model = grid_search.best_estimator_
    y_pred_xgb = best_model.predict(X_test_inliers)

    # Evaluasi hasil prediksi
    mse, rmse, mae, r2 = evaluate_model(y_test_inliers, y_pred_xgb)

    # Simpan hasil Cross-Validation MSE (dikonversi dari negatif)
    cv_mse = -grid_search.best_score_

    # Simpan hasil ke dictionary
    results["Test Size"].append(f"{int(test_size * 100)}%")
    results["CV MSE"].append(round(cv_mse, 4))
    results["MSE"].append(round(mse, 4))
    results["RMSE"].append(round(rmse, 4))
    results["MAE"].append(round(mae, 4))
    results["R²"].append(round(r2, 4))

# Konversi hasil ke dalam DataFrame
results_df = pd.DataFrame(results)

# Tampilkan hasil sebagai tabel yang mudah dibaca
print("\nPrediction Evaluation with XGBoost GridSearchCV Hyperparameter Tuning")
print(results_df.to_string(index=False))



Prediction Evaluation with XGBoost GridSearchCV Hyperparameter Tuning
Test Size  CV MSE  MSE   RMSE    MAE     R²
      20%  0.0034  0.0 0.0001 0.0000 1.0000
      25%  0.0080  0.0 0.0028 0.0012 0.9999
      30%  0.0090  0.0 0.0001 0.0000 1.0000
      35%  0.0096  0.0 0.0028 0.0013 0.9999


#RandomizedSearchCV

In [30]:
# Import necessary libraries
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
from tabulate import tabulate

# Set hyperparameters for tuning in RandomizedSearchCV
param_distributions = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'n_estimators': [50, 100, 200],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.3, 0.5],
    'reg_alpha': [0, 0.01, 0.1],
    'reg_lambda': [0.01, 0.1, 1.0]
}

# Prepare an empty dictionary to store RandomizedSearchCV results
results_random_search = {
    "Test Size": [],
    "CV MSE": [],
    "MSE": [],
    "RMSE": [],
    "MAE": [],
    "R²": []
}

# Simulasi pengujian dengan berbagai test sizes (20%, 25%, 30%, 35%)
test_sizes = [0.2, 0.25, 0.3, 0.35]

for test_size in test_sizes:
    # Split data into train and test sets
    X_train_inliers, X_test_inliers, y_train_inliers, y_test_inliers = train_test_split(X_inliers_test, y_inliers_test, test_size=test_size, random_state=42)

    # Inisialisasi XGBoost Regressor
    xg_reg = XGBRegressor(objective='reg:squarederror')

    # RandomizedSearchCV untuk hyperparameter tuning
    random_search = RandomizedSearchCV(
        estimator=xg_reg,
        param_distributions=param_distributions,
        n_iter=50,  # Jumlah iterasi, dapat diatur lebih besar atau lebih kecil tergantung kebutuhan
        cv=5,  # 5-fold cross-validation
        scoring='neg_mean_squared_error',
        random_state=42,
        n_jobs=-1
    )

    # Fit RandomizedSearchCV model
    random_search.fit(X_train_inliers, y_train_inliers)

    # Predict with the best model found by RandomizedSearchCV
    best_model_random = random_search.best_estimator_
    y_pred_xgb_random = best_model_random.predict(X_test_inliers)

    # Evaluasi hasil prediksi
    mse, rmse, mae, r2 = evaluate_model(y_test_inliers, y_pred_xgb_random)

    # Simpan hasil Cross-Validation MSE (dikonversi dari negatif)
    cv_mse = -random_search.best_score_

    # Simpan hasil ke dictionary
    results_random_search["Test Size"].append(f"{int(test_size * 100)}%")
    results_random_search["CV MSE"].append(round(cv_mse, 4))
    results_random_search["MSE"].append(round(mse, 4))
    results_random_search["RMSE"].append(round(rmse, 4))
    results_random_search["MAE"].append(round(mae, 4))
    results_random_search["R²"].append(round(r2, 4))

# Konversi hasil ke dalam DataFrame
results_random_search_df = pd.DataFrame(results_random_search)

# Tampilkan hasil sebagai tabel yang mudah dibaca
print("\nPrediction Evaluation with XGBoost RandomizedSearchCV Hyperparameter Tuning")
print(results_random_search_df.to_string(index=False))



Prediction Evaluation with XGBoost RandomizedSearchCV Hyperparameter Tuning
Test Size  CV MSE    MSE   RMSE    MAE     R²
      20%  0.0018 0.0002 0.0130 0.0056 0.9991
      25%  0.0045 0.0004 0.0212 0.0098 0.9969
      30%  0.0035 0.0001 0.0085 0.0028 0.9995
      35%  0.0080 0.0001 0.0119 0.0051 0.9990
